# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [34]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib



## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [35]:
ws = Workspace.from_config()
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: udacitymlworkspace
Azure region: eastus
Subscription id: dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f
Resource group: udacityml


In [36]:
cluster_name = "udacityMLCluster"

# Create compute cluster

#verify that cluster does not exist
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', cluster_name)
    compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
    compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 4)

    vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'Standard_D2_V2')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes= compute_min_nodes, max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [37]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "C": uniform(0.5, 1.0),
        "max_iter": choice([5,100])
    }
)

if "training2" not in os.listdir():
    os.mkdir("./training2")


# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
 
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=compute_target,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling = param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [38]:
import pandas as pd
webpath = "https://raw.githubusercontent.com/Ed-Ramos/ML-with-Azure-Capstone/master/heart_failure_clinical_records_dataset.csv"
df = pd.read_csv(webpath)
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [40]:
#TODO: Submit your experiment
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(config=hyperdrive_config)
hyperdrive_run.get_status()
#RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_9f914bfc-70dc-4020-9039-cfed397dfcd7
Web View: https://ml.azure.com/runs/HD_9f914bfc-70dc-4020-9039-cfed397dfcd7?wsid=/subscriptions/dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f/resourcegroups/udacityml/workspaces/udacitymlworkspace&tid=8c9a0d04-e250-4aeb-85f4-1b71f173e263

Streaming azureml-logs/hyperdrive.txt

[2023-03-25T19:03:07.015533][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-25T19:03:07.4138801Z][SCHEDULER][INFO]Scheduling job, id='HD_9f914bfc-70dc-4020-9039-cfed397dfcd7_0' 
[2023-03-25T19:03:07.5743114Z][SCHEDULER][INFO]Scheduling job, id='HD_9f914bfc-70dc-4020-9039-cfed397dfcd7_1' 
[2023-03-25T19:03:07.6382731Z][SCHEDULER][INFO]Scheduling job, id='HD_9f914bfc-70dc-4020-9039-cfed397dfcd7_2' 
[2023-03-25T19:03:07.698869][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-25T19:03:07.7512453Z][SCHEDULER][INFO]Scheduling job, id='HD_9f914bfc-70dc-4020-9039-cfed397dfcd7_3' 
[2023-03-2

{'runId': 'HD_9f914bfc-70dc-4020-9039-cfed397dfcd7',
 'target': 'udacityMLCluster',
 'status': 'Completed',
 'startTimeUtc': '2023-03-25T19:03:06.267423Z',
 'endTimeUtc': '2023-03-25T19:04:39.397517Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '844355ee-07c6-4778-890d-817ce09036ce',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': 'infinite_space_size',
  'score': '0.7676767676767676',
  'best_child_run_id': 'HD_9f914bfc-70dc-4020-9039-cfed397dfcd7_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_9f914bfc-70dc-4020-9039-cfed397dfcd7_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetr

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [41]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [48]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
print(best_run.get_file_names())

{'Regularization Strength:': 0.8366278439762685, 'Max iterations:': 100, 'Accuracy': 0.7676767676767676}
['outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [51]:
#TODO: Save the best model
model = best_run.register_model(model_name='best_hyperdrive', model_path='.')
print(model)

best_run.download_file(best_run.get_file_names()[0], "./outputs/best_hyperdrive.joblib")

#joblib.load('./outputs/model.joblib')

Model(workspace=Workspace.create(name='udacitymlworkspace', subscription_id='dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f', resource_group='udacityml'), name=best_hyperdrive, id=best_hyperdrive:3, version=3, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

